# Sum-check protocol

<a id='contents'></a>
## Contents

* [Setup](#setup)
* [Introduction](#introduction)
* [Definitions, conventions, and notation](#terminology)
* [DeMillo–Lipton–Schwartz–Zippel Lemma](#DLSZ)
* [Sum-check protocol of Lund, Fortnow, Karloff, and Nisan, as described by Thaler](#sum_check)
* [Why V should be convinced if P passes sum check](#why)
* [Complexity and proof size](#complexity)
    * [Verifier's runtime](#verifier_runtime)
    * [Verifier's space complexity](#verifier_space)
    * [Prover's runtime](#prover_runtime)
    * [Prover's space complexity](#prover_space)
    * [Proof size](#proof_size)
* [References](#references)

<a id='setup'></a>
## Setup
↑↑ [Contents](#contents) ↓ [Introduction](#introduction)

In [1]:
from pathlib import Path
import os
import sys

# To use this project, set an environment variable named "ZERO_KNOWLEDGE" on your system
# Determine the project root directory and add it to the Python path
notebook_path = Path(os.getcwd()).resolve()  # Path to the current working directory
project_root = notebook_path.parent          # Parent directory of notebooks, which is the project root

# Add the project root directory to the Python path
sys.path.append(str(project_root))

# The setup module is project_root/scripts/setup.py
from scripts.setup import *


PROJECT DIRECTORY TREE AS AT 2025-01-20 08:40:01

BASE PATH: F:\PROJECTS\INFERENCE-LABS\ZK-NOTEBOOKS

├─ assets/
├─ notebooks/
│  └─ 01-sum-check-protocol.ipynb
│  └─ 02-multilinear-extensions.ipynb
│  └─ 03-arithmetic-circuits.ipynb
│  └─ 04-gkr-protocol.ipynb
├─ scripts/
│  └─ arithmetic_circuits.py
│  └─ gkr_protocol.py
│  └─ info.py
│  └─ multilinear_extensions.py
│  └─ setup.py
│  └─ sum_check.py
│  └─ utils.py
└─ .gitignore
└─ LICENSE
└─ README.md
└─ requirements.txt

PATHS TO FIRST-LEVEL SUBDIRECTORIES STORED IN 'PATH' DICTIONARY

├─ path['scripts'] = F:\projects\inference-labs\zk-notebooks\scripts
├─ path['notebooks'] = F:\projects\inference-labs\zk-notebooks\notebooks
├─ path['circuits'] = F:\projects\inference-labs\zk-notebooks\circuits
├─ path['assets'] = F:\projects\inference-labs\zk-notebooks\assets


<a id='introduction'></a>
## Introduction
↑↑ [Contents](#contents) ↑ [Setup](#setup) ↓ [Definitions, conventions, and notation](#terminology)

This notebook replicates Thaler's [[THA2015]](#tha2015) elegant presentation of the sum-check protocol by Lund, Fortnow, Karloff, and Nisan [[LFKN1992]](#lfkn1992), enriched with additional background information and interactive examples.

<a id='terminology'></a>
## Definitions, conventions, and notation
↑↑ [Contents](#contents) ↑ [Introduction](#introduction) ↓ [DeMillo–Lipton–Schwartz–Zippel Lemma](#DLSZ)

Let $\mathbb{F}$ be a finite field, $v$ a positive integer, and $g$ a $v$-variate polynomial with coefficients in $\mathbb{F}$:
\begin{equation}
 g(X_0,\ldots,X_{v - 1}) = \sum_{e \, \in \, \mathbb{N}^{v}} a_{e} X_0^{e_0}\cdots X_{v - 1}^{e_{v - 1}}, \quad a_e \in \mathbb{F}, \quad e = (e_0,\ldots,e_{v-1}), \tag{1}
\end{equation}
where $a_e = 0$ for all but finitely many $e$ (so the sum is effectively a finite one). The ring of all such polynomials is denoted $\mathbb{F}[X_0,\ldots,X_{v - 1}]$. We use zero-based indexing throughout for consistency with the code and to minimize confusion.

**Definition.** Referring to $(1)$, we call $a_e$ the _coefficients_ of $g$. If $a_e = 0$ for all $e \in \mathbb{N}^v$, then we call $g$ the _zero polynomial_. Suppose $g$ is _not_ the zero polynomial. The _total degree_ of $g$ is 
\begin{equation}
\mathrm{deg}(g) = \max \left\{ e_0 + \cdots + e_{v-1} : a_e \ne 0 \right\}, \tag{2}
\end{equation}
i.e. the highest sum of the exponents of the indeterminates in any single term with a nonzero coefficient. The _degree of indeterminate $X_j$ in $g$_ is 
\begin{equation}
\mathrm{deg}_{X_j}(g) = \max \left\{ e_j : a_e \ne 0 \right\}, \tag{3}
\end{equation}
i.e. the highest power of $X_j$ that appears in any single term with a nonzero coefficient. As the indeterminates are indexed we may denote this by $\mathrm{deg}_j(g)$. We call $g$ _multilinear_ if each $e_j$ in $(1)$ is either $0$ or $1$. In other words, $g$ is multilinear if it is linear in each of its indeterminates separately, but not necessarily simultaneously.

**Remark.** (i) The total degree of a univariate polynomial ($v = 1$) is just its degree. We may use the terms _degree_ and _total degree_ interchangeably. (ii) The zero polynomial is denoted by $0$; context should make it clear when this symbol represents the zero polynomial rather than the zero element of the field. The nonzero constant polynomials are precisely those of total degree $0$. We leave the degree of the zero polynomial undefined. Some authors, however, define it as $-\infty$ for convenience, which simplifies statements like $\mathrm{deg}(g_1 + g_2) \le \max\{\mathrm{deg}(g_1),\mathrm{deg}(g_2)\}$ and $\mathrm{def}(g_1g_2) = \mathrm{deg}(g_1) + \mathrm{deg}(g_2)$, without needing to exclude the zero polynomial as a special case. (iii) On the other hand, if we consider $g$ as a $v$-variate polynomial in $\mathbb{F}[X_0,\ldots,X_{v-1}]$, then $\mathrm{deg}_{j}(X_j)$ is defined and nonnegative for every $j$, $0 \le j \le v - 1$. Some authors might say, for instance, that the degree of $X_2$ in $X_0X_1 + X_1^2$, is undefined. In our case, if we understand the polynomial as belonging to $\mathbf{F}[X_0,X_1,X_2]$, then we'd note that we can multiply each term by $X_2^0$ without affecting the polynomial, whence the degree of $X_2$ in this polynomial is $0$.

---

In the RHS of $(1)$, we may replace a subset of $w$ indeterminates $X_j$ with field elements $r_j$, resulting in a $(v - w)$-variate polynomial, which we denote by substituting the corresponding $r_j$ for each replaced $X_j$ in the LHS. For instance, if $v > 1$ then $g(X_0,r_1,\ldots,r_{v-1})$ denotes the univariate polynomial in the indeterminate $X_0$, obtained by substituting $X_1,\ldots,X_{v - 1}$ with $r_1,\ldots,r_{v - 1}$ in the RHS. For a more explicit example: if $g(X_0,X_1,X_2) = X_0X_1X_2 + X_0X_1^2 + X_2$ then $g(X_0,1,1) = 2X_0 + 1$.

The expression $g(r_0,\ldots,r_{v - 1})$ therefore denotes a constant polynomial in $0$ indeterminates, which we typically identify with the constant itself. We call this the _evalulation_ of $g$ at the point $(r_0,\ldots,r_{v - 1})$. We call $(r_0,\ldots,r_{v-1})$ a _root_ or _zero_ of $g$ if $g(r_0,\ldots,r_{v-1}) = 0$. For example, over $\mathbb{F} = \mathbb{Z}/2\mathbb{Z}$, the evaluation of $g(X_0,X_1,X_2) = X_0X_1X_2 + X_0X_1^2 + X_2$ at the point $(1,1,1)$ is $g(1,1,1) = 1$, so $(1,1,1)$ is not a root of $g$, but $g(0,0,0) = 0$, so $(0,0,0)$ is a root of $g$, as are $(0,1,0)$ and $(1,0,0)$.

**Definition.** We call $f : \mathbb{F}^v \to \mathbb{F}$ a _polynomial function_ if there is a $g \in \mathbb{F}[X_0,\ldots,X_{v - 1}]$ such that $f(x_0,\ldots,x_{v - 1}) = g(x_0,\ldots,x_{v - 1})$ for each $(x_0,\ldots,x_{v - 1}) \in \mathbb{F}^v$. 

**Remark.** The distinctions between constant polynomials and constants, and between polynomials and polynomial functions, may seem pedantic but are particularly important for polynomials over finite fields. If $\mathbb{F}$ has order $q$, then $X^q - X$ is certainly not the zero polynomial in $\mathbb{F}[X]$, but $x \mapsto x^q - x$ is the zero function on $\mathbb{F}$ (by Fermat's little theorem). Polynomials are really just sequences of coefficients that we add and multiply in a certain way. Indeterminates $X_j$ serve as placeholders&mdash;not variables&mdash;to allow us to work with abstract algebraic rules of addition and multiplication. If this remark seems opaque, consider that a univariate polynomial can be represented as a sequence of coefficients $(a_0, a_1, a_2, \ldots)$, where only finitely many $a_i$ are nonzero. Given two polynomials $g = (a_0, a_1, a_2, \ldots)$ and $h = (b_0, b_1, b_2, \dots)$, their product $g \cdot h$ is another sequence of coefficients $(c_0, c_1, c_2, \ldots)$ defined by $c_k = \sum_{i=0}^n a_i b_{k-i}$.

**Example.**

In [3]:
from sum_check import total_degree_example

# Can only handle fields of prime order at present: enter 1 for the exponent k.
total_degree_example()


EXAMPLE 1. The polynomial 

g(X_0, X_1, X_2) = X_0*X_1 + 4*X_0*X_2 + 4*X_1**2 + X_1*X_2 

over GF(5) is a non-multilinear polynomial in 3 indeterminates.

The total degree of g is deg(g) = 2

The degree of X_0 in g is deg_X_0(g) = 1.

The degree of X_1 in g is deg_X_1(g) = 2.

The degree of X_2 in g is deg_X_2(g) = 1.



Another example? (y/n) y


Enter a prime p: 331
Enter an exponent k (the field will have order p**k): 1
Enter your polynomial over GF(331) (e.g. 2*X_0**2 + X_0*X_1*X_2 + X_1*X_4**3 + X_1 + X_3): 2*X_0**2 + X_0*X_1*X_2 + X_1*X_4**3 + X_1 + X_3



EXAMPLE 2. The polynomial 

g(X_0, X_1, X_2, X_3, X_4) = 2*X_0**2 + X_0*X_1*X_2 + X_1*X_4**3 + X_1 + X_3 

over GF(331) is a non-multilinear polynomial in 5 indeterminates.

The total degree of g is deg(g) = 4

The degree of X_0 in g is deg_X_0(g) = 2.

The degree of X_1 in g is deg_X_1(g) = 1.

The degree of X_2 in g is deg_X_2(g) = 1.

The degree of X_3 in g is deg_X_3(g) = 1.

The degree of X_4 in g is deg_X_4(g) = 3.



Another example? (y/n) n


<a id='DLSZ'></a>
## DeMillo–Lipton–Schwartz–Zippel Lemma
↑↑ [Contents](#contents) ↑ [Definitions, conventions, and notation](#terminology) ↓ [Sum-check protocol of Lund, Fortnow, Karloff, and Nisan, as described by Thaler](#sum_check)

This result, commonly referred to as the Schwartz-Zippel lemma, tells us that the roots of a multivariate polynomial over a field are relatively rare. This insight has useful implications; for instance, it suggests that two multivariate polynomials over a field are unlikely to agree at a randomly selected point, which is the crux of the soundness error in the sum-check protocol.

The lemma was first proved jointly by DeMillo and Lipton [[DML1978]](#dml1978), preceding the independent discoveries by both Schwartz [[SCH1979]](#sch1979) and Zippel [[ZIP1980]](#zip1980) around the late 1970s. In fact, for the special case of finite fields—which is the only case relevant here—the result was originally established by Ore in the 1922 [[ORE1922]](#ore1922).

To set up the proof of the Schwartz-Zippel lemma, we begin by stating a foundational proposition upon which the proof relies.

**Proposition.** Let $\mathbb{F}$ be a field and $g \in \mathbb{F}[X]$ a nonzero univariate polynomial over $\mathbb{F}$. If $g$ has degree $d$, then $g$ has at most $d$ roots in $\mathbb{F}$. 

_Proof outline_. Suppose $r_0 \in \mathbb{F}$ is a root of $g$, so $g(r_0) = 0$. By the division algorithm, we can factor $g(X) = (X - r_0) q(X)$, where $q(X) \in \mathbb{F}[X]$ has degree $d - 1$. If $r_0$ is a repeated root, then we can factor $g$ further as $g(X) = (X - r_0)^{m_0} q_0(X)$, where $m_0 \geq 1$ is maximal (i.e., $q_0(r_0) \neq 0$). At this point, $q_0(X)$ is a polynomial of degree $d - m_0$. Now, if there is another root $r_1 \neq r_0$, then $g(r_1) = 0$ implies $q_0(r_1) = 0$ because $(r_1 - r_0)^{m_0} \neq 0$ (fields have no zero divisors). Therefore, $q_0(X)$ must also be divisible by $(X - r_1)$, allowing us to write $g(X) = (X - r_0)^{m_0} (X - r_1) q_1(X)$ for some polynomial $q_1(X)$ of degree $d - m_0 - 1$. We can repeat this process, factoring out distinct roots until we have exhausted the degree $d$. Since each distinct root reduces the degree of the remaining polynomial by at least $1$, the number of roots (counted with multiplicity) cannot exceed $d$. Thus, $g$ can have at most $d$ roots in $\mathbb{F}$, counted with multiplicity. This argument also generalizes to any integral domain $R$, where the absence of zero divisors ensures that a similar factorization process holds.

**Proposition [Schwartz-Zippel lemma].** Let $g$ be a $v$-variate ($v \ge 1$), nonzero polynomial of total degree $d$ over a finite field $\mathbb{F}$. We have 
\begin{equation}
\#\{\mathbf{r} \in \mathbb{F}^v : g(\mathbf{r}) = 0\} \le (\#\mathbb{F})^{v - 1}d.
\end{equation}
In other words, if $r_0,\ldots,r_{v - 1}$ are selected at random independently and uniformly from $\mathbb{F}$, then 
\begin{equation}
 \mathrm{Pr}[g(r_0,\ldots,r_{v - 1}) = 0] \le \frac{d}{\# \mathbb{F}}. \tag{4}
\end{equation}

_Proof_. The proof is by induction on $v$. The base case $v = 1$ is the previous proposition. Let $v \ge 1$ and assume the statement of the Schwartz-Zippel lemma holds for all nonzero polynomials in $u$ indeterminates, $1 \le u \le v$. 

Writing $g$ as in $(1)$, note that 
\begin{align*}
g(X_0,\ldots,X_{v - 1}) = \sum_{e \, \in \, \mathbb{N}^{v}} a_{e} X_0^{e_0}\cdots X_{v - 1}^{e_{v - 1}}
= \sum_{e_{v-1} \, \in \, \mathbb{N}} X_{v-1}^{e_{v-1}} \sum_{(e_0,\ldots,e_{v-2}) \, \in \, \mathbb{N}^{v-1}} a_{e} X_0^{e_0}\cdots X_{v - 2}^{e_{v - 2}}  = \sum_{k \, = \, 0}^{d} X_{v-1}^k h_i(X_0,\ldots,X_{v-2}),
\end{align*}
say, where we have replaced $e_{v - 1}$ by $k$ in the outer sum and truncated it at $k = d$, because $g$ is assumed ot have degree $d$ and so $a_e = 0$ whenever $e_{v - 1} > d$. We can't have $h_k \equiv 0$ for every $k$, for in that case we'd also have $g \equiv 0$. Let $j = \max\{k: h_k \not\equiv 0\}$ be the largest $k$ for which $h_k$ is a nonzero polynomial. We partition the set of roots of $g$ according to whether or not the first $v - 1$ coordinates also constitute a root of $h_j$:
\begin{align*}
\{\mathbf{r} \in \mathbb{F}^{v} : g(\mathbf{r}) = 0\}
 \subseteq \{\mathbf{r} \in \mathbb{F}^{v} : h_j(r_0,\ldots,r_{v-2}) = 0\} \cup \{\mathbf{r} \in \mathbb{F}^v :  h_j(r_0,\ldots,r_{v-2}) \ne 0 \,\, \text{and} \,\, g(\mathbf{r}) = 0 \}.
\end{align*}
We will show that the sets on the RHS have sizes bounded by $(\#\mathbb{F})^{v - 1}(d - j)$ and $(\#\mathbb{F})^{v - 1}j$ respectively.

Since $X_{v-1}^jh_j(X_0,\ldots,X_{v-2})$ has degree at most $\mathrm{deg} \, (g) = d$, we have $\mathrm{deg} \, (h_j) \le d - j$. Therefore, by inductive hypothesis, 
\begin{equation}
 \#\{(r_0,\ldots,r_{v - 2}) \in \mathbb{F}^{v - 1} : h_j(r_0,\ldots,r_{v-2}) = 0\} \le (\#\mathbb{F})^{v - 2}(d - j).
\end{equation}
Consequently,
\begin{equation}
 \#\{(r_0,\ldots,r_{v - 2},r_{v-1}) \in \mathbb{F}^v : h_j(r_0,\ldots,r_{v-2}) = 0\} \le (\#\mathbb{F})^{v - 1}(d - j).
\end{equation}

If $h_j(r_0,\ldots,r_{v - 2}) \ne 0$, then $g(r_0,\ldots,r_{v-2},X_{v-1})$ is a nonzero univariate polynomial of degree $j$ (its leading term is $X_{v-1}^jh_j(r_0,\ldots,r_{v-2}))$. There are trivially at most $(\#\mathbb{F})^{v-1}$ tuples $(r_0,\ldots,r_{v-2}) \in \mathbb{F}^{v - 1}$ such that $h_j(r_0,\ldots,r_{v-2}) \ne 0$, and for each such tuple, there are at most $j$ elements $r_{v - 1}$ of $\mathbb{F}$ such that $g(r_0,\ldots,r_{v-1}) = 0$. This is by inductive hypothesis, because such $r_{v-1}$ are roots of $g(r_0,\ldots,r_{v-2},X_{v-1})$. Hence,
\begin{equation}
\#\{(r_0,\ldots,r_{v - 1}) \in \mathbb{F}^v :  h_j(r_0,\ldots,r_{v-2}) \ne 0 \,\, \text{and} \,\, g(r_0,\ldots,r_{v-1}) = 0 \} \le (\#\mathbb{F})^{v-1}j.
\end{equation}

**Remark**. (i) The proof shows that $\mathbb{F}$ can actually be replaced by any integral domain $R$ (whether finite or infinite), provided we select $r_0, \ldots, r_{v - 1}$ from a finite subset $S \subseteq R$ and substitute $\#S$ for $\#\mathbb{F}$ in the denominator on the right-hand side. This generalization—the full result established by DeMillo and Lipton, Schwartz, and Zippel—is not required for our current purposes. (ii) Some people mistakenly refer to the base case as the much deeper fundamental theorem of algebra, which states that a non-zero polynomial of degree $d$ over $\mathbb{C}$ has exactly $d$ roots in $\mathbb{C}$, counted with multiplicity.

**Example.**

In [4]:
from sum_check import roots_example

# Can only handle fields of prime order at present: enter 1 for the exponent k.
roots_example(time_out=60)


EXAMPLE 1. The polynomial X_0*X_1 + X_2**2 has 1681 roots in GF(41)**(3).

Thus, if r_0, r_1, r_2 are chosen independently and uniformly at random from GF(41), then g(r_0, r_1, r_2) = 0 with probability 1681/41**(3) = 0.024390243902439025.

The polynomial has total degree d = 2 and 2/41 = 0.04878048780487805.

Here are 10 of the roots:

g(27, 35, 11) = 0
 g(24, 19, 6) = 0
 g(6, 11, 37) = 0
  g(0, 34, 0) = 0
 g(11, 6, 37) = 0
g(34, 13, 38) = 0
  g(5, 4, 29) = 0
  g(1, 33, 7) = 0
g(11, 35, 36) = 0
 g(24, 13, 4) = 0



Another example? (y/n) y
Enter a prime p: 331
Enter an exponent k (the field will have order p**k): 1
Enter your polynomial over GF(331) (e.g. 2*X_0**2 + X_0*X_1*X_2 + X_1*X_4**3 + X_1 + X_3): 2*X_0**2 + X_0*X_1*X_2 + X_1*X_4**3 + X_1 + X_3



EXAMPLE 2. In 60 seconds, we found 7944 roots of 2*X_0**2 + X_0*X_1*X_2 + X_1*X_4**3 + X_1 + X_3 in GF(331)**(5).

There may be more roots, so what follows is not a verification of Schwartz-Zippel.

Thus, if r_0, r_1, r_2, r_3, r_4 are chosen independently and uniformly at random from GF(331), then g(r_0, r_1, r_2, r_3, r_4) = 0 with probability at least 7944/331**(5) = 1.999398061053123e-09.

The polynomial has total degree d = 4 and 4/331 = 0.012084592145015106.

Here are 10 of the roots:

g(0, 0, 20, 0, 161) = 0
 g(0, 0, 9, 0, 291) = 0
 g(0, 0, 3, 0, 269) = 0
 g(0, 0, 23, 0, 41) = 0
  g(0, 0, 3, 0, 40) = 0
 g(0, 0, 6, 0, 149) = 0
 g(0, 0, 22, 0, 10) = 0
g(0, 0, 16, 0, 112) = 0
 g(0, 0, 10, 0, 90) = 0
 g(0, 0, 14, 0, 20) = 0



Another example? (y/n) n


<a id='sum_check'></a>
## Sum-check protocol of Lund, Fortnow, Karloff, and Nisan, as described by Thaler
↑↑ [Contents](#contents) ↑ [DeMillo–Lipton–Schwartz–Zippel Lemma](#DLSZ) ↓ [Why V should be convinced if P passes sum check](#why)

This is a description of the sum-check protocol as presented by Lund, Fortnow, Karloff, and Nisan in [[LFKN1992]](#lfkn1992), with the exposition following Thaler’s treatment in [[THA2015]](#tha2015).

**Initialization.** Given a nonzero $v$-variate polynomial $g$ over a finite field $\mathbb{F}$, let 
\begin{equation}
 H := \sum_{b_0 \in \{0,1\}} \cdots \sum_{b_{v-1} \in \{0,1\}} g(b_0,\ldots,b_{v - 1}). \tag{5}
\end{equation}
Although verifier V does not know $g$, the number of indeterminates $v$ and the degree $d_j$ in each of its indeterminates $X_j$ are known to V. Prover P claims that $H = H^*$, sending this claim to V, who needs to verify that $H = H^*$ by interacting with P. 

**Round 0.** Let 
\begin{equation}
g_0(X_0) := \sum_{b_1 \in \{0,1\}} \cdots \sum_{b_{v-1} \in \{0,1\}} g(X_0,b_1,\ldots,b_{v - 1}). \tag{6}
\end{equation}
P sends polynomial $g^*_0$ to V, claiming that $g^*_0 = g_0$. If so, then 
\begin{equation*}
 \mathrm{deg}(g^*_0) = \mathrm{deg}(g_0) \le \mathrm{deg}_0(g) = d_0.
\end{equation*}
V verifies this: if $\mathrm{deg}(g^*_0) > d_0$ then V 'rejects' and the protocol terminates. Otherwise, we continue. If P's claims that $H = H^*$ and $g^*_0 = g_0$ are both true, then 
\begin{equation}
H^* = g^*_0(0) + g^*_0(1). \tag{7}
\end{equation}
This is because $H = g_0(0) + g_0(1)$. V verifies ($7$), which is easy because $g^*_0$ is a univariate polynomial of degree at most $d$. If ($7$) does not hold, then V rejects and the protocol terminates. Otherwise, V chooses $r_0$ uniformly at random from $\mathbb{F}$, and sends it to P.

**Round j** ($1 \le j \le v - 1$). Let 
\begin{equation}
g_j(X_j) := \sum_{b_{j+1} \in \{0,1\}} \cdots \sum_{b_{v-1} \in \{0,1\}} g(r_0,r_1,\ldots,r_{j-1},X_j,b_{j+1},\ldots,b_{v - 1}). \tag{8}
\end{equation}
P responds to V's random challenge $r_{j - 1}$ by sending V a polynomial $g^*_{j}$, claiming that $g^*_{j} = g_j$. If so, then 
\begin{equation*}
 \mathrm{deg}(g^*_j) = \mathrm{deg}(g_j) \le \mathrm{deg}_j(g) = d_j.
\end{equation*}
V verifies this: if $\mathrm{deg}(g^*_j) > d_j$ then V 'rejects' and the protocol terminates. Otherwise, we continue. If P's previous claims that $g^*_{j-1} = g_{j - 1}$ and $g^*_{j} = g_{j}$ are both true, then 
\begin{align*}
g^*_{j - 1}(r_{j - 1}) = g^*_j(0) + g^*_j(1). \tag{9}
\end{align*}
This is because $g_{j - 1}(r_{j - 1}) = g_j(0) + g_j(1)$. To see this, note that by definition ($8$),
\begin{align*}
g_{j - 1}(r_{j - 1}) & = \sum_{b_{j} \in \{0,1\}} \cdots \sum_{b_{v-1} \in \{0,1\}} g(r_0,r_1,\ldots,r_{j-2},r_{j-1},b_{j},\ldots,b_{v - 1})
\end{align*}
and 
\begin{align*}
g_j(0) + g_j(1) & = \sum_{b_{j+1} \in \{0,1\}} \cdots \sum_{b_{v-1} \in \{0,1\}} g(r_0,r_1,\ldots,r_{j-1},0,b_{j+1},\ldots,b_{v - 1})
 + \sum_{b_{j+1} \in \{0,1\}} \cdots \sum_{b_{v-1} \in \{0,1\}} g(r_0,r_1,\ldots,r_{j-1},1,b_{j+1},\ldots,b_{v - 1}) \\
 & = \sum_{b_{j} \in \{0,1\}} \cdots \sum_{b_{v-1} \in \{0,1\}} g(r_0,r_1,\ldots,r_{j-1},b_j,b_{j+1},\ldots,b_{v - 1}).
\end{align*}
V verifies $(9)$, which is easy because $g_{j - 1}$ and $g_{j}$ are univariate polynomials of degree at most $d$. If ($9$) does not hold, V rejects and the protocol terminates. Otherwise, V chooses $r_j$ uniformly at random from $\mathbb{F}$, independently of $r_0,\ldots,r_{j-1}$, and sends it to P.

It helps to write out Round $v - 1$ explicitly. In the final round, P sends $g^*_{v - 1}$ to V, claiming that $g^*_{v - 1} = g_{v - 1}$, where
\begin{equation}
g_{v - 1}(X_{v - 1}) := g(r_0,\ldots,r_{v - 2},X_{v - 1}). \tag{10}
\end{equation}
V verifies that $\mathrm{deg}(g_{v-1}) \le d_{v-1}$ and that $g^*_{v - 2}(r_{v - 2}) = g^*_{v - 1}(0) + g^*_{v - 1}(1)$. If either the degree bound or ($10$) does not hold, V rejects and the protocol terminates. Otherwise, we continue.

**Final check.** V chooses $r_{v - 1}$ uniformly at random from $\mathbb{F}$, independently of $r_0,\ldots,r_{v-2}$, and verifies that 
\begin{equation}
g^*_{v - 1}(r_{v - 1}) = g(r_0,\ldots,r_{v - 2},r_{v - 1}), \tag{11}
\end{equation}
which holds if $g^*_{v-1} = g_{v-1}$, in view of $(10)$. If ($11$) does not hold, then V rejects. Otherwise, V accepts that $H = H^*$ and P passes the sum-check protocol. The protocol is complete. 

Note that V isn't supposed to know $g$, nor is $V$ necessarily capable of efficiently evaluating a $v$-variate polynomial at a point. V obtains the RHS of $(11)$ either from P, in which case P must have made a 'commitment' to $g$ at the outset, or from an 'oracle'.

**Example.**

In [5]:
from sum_check import sum_check_example

# Can only handle fields of prime order at present: enter 1 for the exponent k.
sum_check_example();

EXAMPLE 1. 

INITIALIZATION

Let

g(X_0, X_1, X_2) = X_0*X_1 + 4*X_0*X_2 + 4*X_1**2 + X_1*X_2.

V does not know g, but does know that it is a polynomial over GF(5) in 3 indeterminates.

V also knows an upper bound for the degree of each indeterminate in g.

Let

H := sum g(b_0, b_1, b_2) over (b_0, b_1, b_2) in {0,1}**3.

P claims that and H = H*, where H* = 3.

ROUND 0

Let

g_0(X_0) := sum g(X_0, b_1, b_2) over (b_1, b_2) in {0,1}**2

P claims that g_0(X_0) = g*_0(X_0), where 

g*_0(X_0) = 4.

If P's last two claims are true, then deg(g*_0) ≤ deg_0(g) and H* = g*_0(0) + g*_0(1).

V first verifies that deg(g*_0) = 0, which is indeed less than or equal to deg_0(g) = 1.

P's claim is that H = H* = 3. V proceeds to compute:

g*_0(0) + g*_0(1) = 3

This is easy because g*_0 is univariate of degree at most 1.

V sees that P's claims so far are consistent.

V selects 4 uniformly at random from GF(5), independently of any previous choices.

ROUND 1

Let

g_1(X_1) := sum g(4, X_1, b_2) over (


Another example? (y/n) y


EXAMPLE 2. 


Enter a prime p: 331
Enter an exponent k (the field will have order p**k): 1
Enter your polynomial over GF(331) (e.g. 2*X_0**2 + X_0*X_1*X_2 + X_1*X_4**3 + X_1 + X_3): 2*X_0**2 + X_0*X_1*X_2 + X_1*X_4**3 + X_1 + X_3

Do you want this to be interactive? (y/n):  n
Choose a, b, or c: Prover (a) will lie. (b) will lie with probability 1/2. (c) will not lie. c



INITIALIZATION

Let

g(X_0, X_1, X_2, X_3, X_4) = 2*X_0**2 + X_0*X_1*X_2 + X_1*X_4**3 + X_1 + X_3.

V does not know g, but does know that it is a polynomial over GF(331) in 5 indeterminates.

V also knows an upper bound for the degree of each indeterminate in g.

Let

H := sum g(b_0, b_1, b_2, b_3, b_4) over (b_0, b_1, b_2, b_3, b_4) in {0,1}**5.

P claims that and H = H*, where H* = 76.

ROUND 0

Let

g_0(X_0) := sum g(X_0, b_1, b_2, b_3, b_4) over (b_1, b_2, b_3, b_4) in {0,1}**4

P claims that g_0(X_0) = g*_0(X_0), where 

g*_0(X_0) = 32*X_0**2 + 4*X_0 + 20.

If P's last two claims are true, then deg(g*_0) ≤ deg_0(g) and H* = g*_0(0) + g*_0(1).

V first verifies that deg(g*_0) = 2, which is indeed less than or equal to deg_0(g) = 2.

P's claim is that H = H* = 76. V proceeds to compute:

g*_0(0) + g*_0(1) = 76

This is easy because g*_0 is univariate of degree at most 2.

V sees that P's claims so far are consistent.

V selects 247 uniformly at random from GF(331), independently of


Another example? (y/n) n


<a id='why'></a>
## Why V should be convinced if P passes sum check
↑↑ [Contents](#contents) ↑ [Sum-check protocol of Lund, Fortnow, Karloff, and Nisan, as described by Thaler](#sum_check) ↓ [Complexity and proof size](#complexity)

The sum-check protocol is _complete_: if P's claims are true, P will pass the protocol, assuming V's calculations are accurate. In other words, if P fails the protocol, P is dishonest (their claims are inconsistent).

In terms of _soundness_, a dishonest P may provide an incorrect value for the initial sum $H$ and still pass sum check. However, we can make the probability of this event—the soundness error—as small as desired by choosing a large field size $\#\mathbb{F}$ relative to $vd$. The probability of V being convinced of a false value for the initial sum is at most $vd/\#\mathbb{F}$.


To recap the sum-check protocol, we have 
\begin{align*}
 H & := \sum_{b_0 \, \in \, \{0,1\}} \cdots \sum_{b_{v - 1} \, \in \, \{0,1\}} g(X_0,\ldots,X_{v-1}) \\
 g_0(X_0) & := \sum_{b_1 \, \in \, \{0,1\}} \cdots \sum_{b_{v - 1} \, \in \, \{0,1\}} g(X_0,b_1,\ldots,b_{v-1}) 
\end{align*}
Also, $r_0,\ldots,r_{v-1}$ are selected independently and uniformly at random from $\mathbb{F}$, and 
\begin{align*}
 g_j(X_j) & := \sum_{b_{j + 1} \, \in \, \{0,1\}} \cdots \sum_{b_{v - 1} \, \in \, \{0,1\}} g(r_0,\ldots,r_{j-1},X_j,b_{j+1},\ldots,b_{v-1}), \quad 0 \le j \le v - 2 \\
 g_{v-1}(X_{v-1}) & := g(r_0,\ldots,r_{v-2},X_{v-1})
\end{align*}
Letting $d_j := \mathrm{deg}_{j}(g)$ for $0 \le j \le v - 1$, we have 
\begin{equation}
 \mathrm{deg}(g_j) \le d_j, \quad 0 \le j \le v - 1.
\end{equation}
Also,
\begin{align*}
 H & = g_0(0) + g_0(1) \\
 g_{j-1}(r_{j-1}) & = g_j(0) + g_j(1), \quad 1 \le j \le v - 1 \\
 g_{v-1}(r_{v-1}) & = g(r_0,\ldots,r_{v-1})
\end{align*}
P claims that $H = H^*$ and $g_j = g_j^*$ for $0 \le j \le v - 1$, passing sum check if and only if the V's acceptance conditions hold:
\begin{align*}
\tag{$*$}
\begin{split}
\mathrm{deg}(g^*_j) & \le d_j, \quad 0 \le j \le v - 1, \\
 H^* & = g^*_0(0) + g^*_0(1) \\
 g^*_{j-1}(r_{j-1}) & = g^*_j(0) + g^*_j(1), \quad 1 \le j \le v - 1 \\
 g^*_{v-1}(r_{v-1}) & = g(r_0,\ldots,r_{v-1}) 
 \end{split}
\end{align*}

Suppose P's claim for the value of the initial sum is false, i.e. in fact $H^* \ne H$, and that P passes sum check in spite of this, i.e. V's acceptance conditions $(*)$ all hold. If we had $g^*_0 = g_0$, then we'd have $g^*(0) + g^*(1) = g(0) + g(1) = H \ne H^*$, and so V would reject in Round $0$. Therefore, $g^*_0 \ne g_0$. 

**Case 1.** $g^*_j \ne g_j$ for all $j$, $0 \le j \le v - 1$. In particular, $g^*_{v - 1} \ne g_{v - 1}$, and by the final acceptance condition,
\begin{equation*}
g^*_{v-1}(r_{v - 1}) = g_{v-1}(r_{v-1}) = g(r_0,\ldots,r_{v-1}).
\end{equation*}

**Case 2.** There is a minimal $k$, $1 \le k \le v - 1$, such that $g^*_{k-1} \ne g_{k-1}$ and $g^*_k = g_k$. By the acceptance condition of Round $k$,
\begin{equation*}
g^*_{k-1}(r_{k-1}) = g^*_k(0) + g^*_k(1) = g_k(0) + g_k(1) = g_{k - 1}(r_{k - 1}).
\end{equation*}


In either case, there is a sequence of polynomials $g^*_0 \ne g_0,\ldots,g^*_{k-1} \ne g^*_{k-1}$, where $1 \le k \le v$, such that $g^*_{k-1}(r_{k-1}) = g_{k-1}(r_{k-1})$. Since $g_{k - 1}$ and $g^*_{k-1}$ are chosen _before_ $r_{k - 1}$, they are independent of $r_{k - 1}$. Also, by the degree bounds, all polynomials have degree at most $d := \mathrm{deg}(g)$. For each $k$, $1 \le k \le v$, the probability of this event is therefore at most
\begin{equation*}
 \mathrm{Pr}[(g^*_{k - 1} - g_{k - 1})(r_{k-1}) = 0] \le \frac{d}{\#\mathbb{F}} \tag{12}
\end{equation*}
by the Schwartz-Zippel lemma, because $g^*_{k - 1} - g_{k - 1}$ is a nonzero polynomial of degree at most $d$, and $r_{k-1}$ is chosen uniformly at random from $\mathbb{F}$.

In each round $j \ge 1$, while V verifies that $g^*_{j-1}(r_{j-1}) = g^*_j(0) + g^*_j(1)$ (which could hold even if $g^*_{j-1}(r_{j-1}) \ne g_{j-1}(r_{j-1})$), P can check whether $g^*_{j - 1}(r_{j - 1}) = g_{j-1}(r_{j - 1})$. If this equality holds, P can then revert to sending the 'true' polynomials $g^*_j = g_j$ for subsequent rounds $j$ and pass the sum-check protocol.

Finally, we note that V will accept a false value for the sum $H$ in the end in the event that the above occurs for any $k$, $1 \le k \le v$. Therefore, by the union bound and $(12)$, 
\begin{equation}
 \mathrm{Pr}[ \text{V accepts} \, | \, H^* \ne H] \le \frac{vd}{\#\mathbb{F}}. \tag{13}
\end{equation}

**Remark.** (i) If P provides a false value $H^*$ for the initial sum, their strategy to pass the sum-check protocol might involve finding a polynomial $g^*_0 \ne g_0$, of degree at most $d_0$, such that $g^*_0 - g_0$ has as many roots as possible in $\mathbb{F}$, while satisfying the condition $g^*_0(0) + g^*_0(1) = H^*$. This maximizes the likelihood that $g^*_0(r_0) = g_0(r_0)$. If this coincidence occurs, P can immediately switch to sending the 'true' polynomials to V, starting with $g^*_1 = g_1$, since the acceptance condition $g^*_0(r_0) = g^*_1(0) + g^*_1(1)$ will hold (because $g_0(r_0) = g_1(0) + g_1(1)$). Notably, the acceptance condition might still hold even if $g^*_0(r_0) \ne g_0(r_0)$. In that case, P would seek a polynomial $g^*_1 \ne g_1$ such that $g^*_1 - g_1$ has as many roots as possible in $\mathbb{F}$, and so on.

For instance, suppose $\mathbb{F}$ is a prime field of order $p$, and that $g_0$ has degree $d_0$, where $2 \le d_0 < p$. If $\{\alpha_0,\ldots,\alpha_{k-1}\}$ is any set of $k$ distinct field elements such that 
\begin{equation*}
[(0 - \alpha_0)\cdots (0 - \alpha_{k-1})] + [(1 - \alpha_0)\cdots (1 - \alpha_{k-1})] \ne 0,
\end{equation*}
and if $\alpha_{k} \in \mathbb{F}$ satisfies
\begin{equation*}
 (0 - \alpha_{k}) = \frac{[H^* - H] - [(1 - \alpha_0)\cdots (1 - \alpha_{k-1})]}{[(0 - \alpha_0)\cdots (0 - \alpha_{k-1})] + [(1 - \alpha_0)\cdots (1 - \alpha_{k-1})]},
\end{equation*}
then 
\begin{equation*}
[(0 - \alpha_0)\cdots (0 - \alpha_{k})] + [(1 - \alpha_0)\cdots (1 - \alpha_{k})] = H^* - H \ne 0.
\end{equation*}
If $\alpha_{k} \not\in \{\alpha_0,\ldots,\alpha_{k - 1}\}$, then we now have $k + 1$ distinct field elements $\alpha_0,\ldots,\alpha_k$ such that 
\begin{equation*}
g^*_0(X_0) := g_0(X_0) + (X_0 - \alpha_0)\cdots (X_0 - \alpha_{k})
\end{equation*}
satisfies 
\begin{equation*}
g^*_0(0) + g^*(1) = H^*.
\end{equation*}
A dishonest prover P may try to generate such a polynomial $g^*_0$ with $k$ as large as possible while not exceeding $d_0 - 1$. Then $g_0^*$ is distinct from $g_0$, has degree at most $d_0$, and $g^*_0(r_0) = g_0(r_0)$ if and only if $r_0 \in \{\alpha_0,\alpha_1,\ldots,\alpha_{k}\}$. That is, $\mathrm{Pr}[ g^*_0(r_0) = g_0(r_0) ] = (k+1)/\#\mathbb{F}$. Given that $g^*_0 \ne g_0$ and $\mathrm{deg}(g^*_0) \le \mathrm{deg}(g_0) \le \mathrm{deg}(g)$, this probability can be at most $d/\#\mathbb{F}$, by the Schwartz-Zippel lemma.

Alternatively, P could start with a multivariate polynomial $g^*$ different from $g$, where $\mathrm{deg}(g^*) \le d$ and the sum of $g^*$ over the Boolean hypercube equals $H^* \ne H$. P would then respond to all of V's challenges as though $g^*$ were the same as $g$. For the final check to pass, it would require that $g^*(r_0, \ldots, r_{v-1}) = g(r_0, \ldots, r_{v-1})$. P could maximize the likelihood of this by choosing a $g^* \ne g$ such that $g^* - g$ has as many roots as possible in $\mathbb{F}^v$. By the Schwartz-Zippel lemma, the probability that $(r_0, \ldots, r_{v - 1})$ is a root of $g^* - g$ remains at most $vd/\#\mathbb{F}$.

(ii) If the degree $d$ of $g$ were concealed from V, a dishonest P could inflate the degrees of the $g^*_j$ polynomials, increasing the probability of passing.

(iii) In the sum-check protocol, the degree $d$ of $g$ and the number of indeterminates $v$ are typically known to V beforehand, as they are determined by the structure of the problem being verified. These parameters are either agreed upon during the setup phase or are intrinsic to the function being evaluated (e.g., a [multilinear extension](./02-multilinear_extensions) or arithmetic circuit). The verifier ensures the prover's adherence to these parameters by checking that the submitted polynomials meet the degree constraints at each round.


In [7]:
from sum_check import sum_check

sum_check();

Enter a prime p: 331
Enter an exponent k (the field will have order p**k): 1
Enter your polynomial over GF(331) (e.g. 2*X_0**2 + X_0*X_1*X_2 + X_1*X_4**3 + X_1 + X_3): 2*X_0**2 + X_0*X_1*X_2 + X_1*X_4**3 + X_1 + X_3

Do you want this to be interactive? (y/n):  y
Do you want to act as verifier? (y/n):  y
Do you want to act as prover? (y/n):  y



INITIALIZATION

Let

g(X_0, X_1, X_2, X_3, X_4) = 2*X_0**2 + X_0*X_1*X_2 + X_1*X_4**3 + X_1 + X_3.

V does not know g, but does know that it is a polynomial over GF(331) in 5 indeterminates.

V also knows an upper bound for the degree of each indeterminate in g.

Let

H := sum g(b_0, b_1, b_2, b_3, b_4) over (b_0, b_1, b_2, b_3, b_4) in {0,1}**5.



As prover, enter your claim H* for the value of H (the true value is 76): 0



ROUND 0

Let

g_0(X_0) := sum g(X_0, b_1, b_2, b_3, b_4) over (b_1, b_2, b_3, b_4) in {0,1}**4

=== START INVISIBLE TO VERIFIER ===

In fact, 

g_0(X_0) = 32*X_0**2 + 4*X_0 + 20.

Your goal is to convince V that the polynomial g*_0 you're about to send is g_0 (a false claim).

V will check that:

(a) deg(g*_0) ≤ deg_0(g) = 2

(b) g*_0(0) + g*_0(1) = 0 (which implies g*_0 ≠ g_0, but V doesn't know this)

If it turns out that

(c) g*_0(r_0) = g_0(r_0), where r_0 is the random element of GF(331) that V will choose after receiving g*_0,

then V will accept your original claim if you send the 'true' polynomials g*_k = g_k in subsequent rounds.

As you don't know what r_0 will be, the best you can do is choose g*_0 satisfying (a) and (b) such that

(d) g*_0 - g_0 has as many roots as possible in GF(331) (there can be at most deg(g*_0 - g_0) ≤ 2.

For example, if

g*_0(X_0) = 33*X_0**2 + 258*X_0 + 20,

then (a) and (b) hold, and the roots (in GF(331)) of g*_0 - g_0 are: 0, 77.

The probabili


Enter your g*_0(X_0): 33*X_0**2 + 258*X_0 + 20


...


V selects 183 uniformly at random from GF(331), independently of any previous choices.

=== START INVISIBLE TO VERIFIER ===

YOU HAVE SUCCESSFULLY DECEIVED THE VERIFIER: g*_3(183) = g_3(183)

From now on, the g*_j will automatically be set to g_j.

=== END INVISIBLE TO VERIFIER ===

ROUND 3

Let

g_3(X_3) := sum g(1, 44, 183, X_3, b_4) over (b_4) in {0,1}**1

P claims that g_3(X_3) = g*_3(X_3), where 

g*_3(X_3) = 2*X_3 + 21.

If P's last two claims are true, then deg(g*_3) ≤ deg_3(g) and g*_2(183) = g*_3(0) + g*_3(1).

V first verifies that deg(g*_3) = 1, which is indeed less than or equal to deg_3(g) = 1.

V proceeds to compute:

        g*_2(183) = 44
g*_3(0) + g*_3(1) = 44

This is easy because the g*_2 and g*_3 are univariate of degree at most 1.

V sees that P's claims so far are consistent.



As verifier, choose an element of GF(331) uniformly at random, independent of any previous choice: 1



V selects 1 uniformly at random from GF(331), independently of any previous choices.

ROUND 4

Let

g_4(X_4) := g(1, 44, 183, 1, X_4)

P claims that g_4(X_4) = g*_4(X_4), where 

g*_4(X_4) = 44*X_4**3 + 155.

If P's last two claims are true, then deg(g*_4) ≤ deg_4(g) and g*_3(1) = g*_4(0) + g*_4(1).

V first verifies that deg(g*_4) = 3, which is indeed less than or equal to deg_4(g) = 3.

V proceeds to compute:

          g*_3(1) = 23
g*_4(0) + g*_4(1) = 23

This is easy because the g*_3 and g*_4 are univariate of degree at most 3.

V sees that P's claims so far are consistent.



As verifier, choose an element of GF(331) uniformly at random, independent of any previous choice: 4



FINAL CHECK

V selects 4 uniformly at random from GF(331), independently of any previous choices.

If all of P's claims are true, then

g*_4(4) = g(1, 44, 183, 1, 4).

V computes

g*_4(4) = 323.

Finally, V appeals to an oracle---or P, if P has made a commitment to g---to determine that

g(1, 44, 183, 1, 4) = 323.

P passes the final verification, and V ACCEPTS P's initial claim that H = H*.

P HAS DECEIVED V: THE TRUE VALUE OF H IS IN FACT 76.


In [8]:
sum_check();

Enter a prime p: 331
Enter an exponent k (the field will have order p**k): 1
Enter your polynomial over GF(331) (e.g. 2*X_0**2 + X_0*X_1*X_2 + X_1*X_4**3 + X_1 + X_3): 2*X_0**2 + X_0*X_1*X_2 + X_1*X_4**3 + X_1 + X_3

Do you want this to be interactive? (y/n):  n
Choose a, b, or c: Prover (a) will lie. (b) will lie with probability 1/2. (c) will not lie. a



INITIALIZATION

Let

g(X_0, X_1, X_2, X_3, X_4) = 2*X_0**2 + X_0*X_1*X_2 + X_1*X_4**3 + X_1 + X_3.

V does not know g, but does know that it is a polynomial over GF(331) in 5 indeterminates.

V also knows an upper bound for the degree of each indeterminate in g.

Let

H := sum g(b_0, b_1, b_2, b_3, b_4) over (b_0, b_1, b_2, b_3, b_4) in {0,1}**5.

P claims that and H = H*, where H* = 2.

ROUND 0

Let

g_0(X_0) := sum g(X_0, b_1, b_2, b_3, b_4) over (b_1, b_2, b_3, b_4) in {0,1}**4

P claims that g_0(X_0) = g*_0(X_0), where 

g*_0(X_0) = 33*X_0**2 + 260*X_0 + 20.

If P's last two claims are true, then deg(g*_0) ≤ deg_0(g) and H* = g*_0(0) + g*_0(1).

V first verifies that deg(g*_0) = 2, which is indeed less than or equal to deg_0(g) = 2.

P's claim is that H = H* = 2. V proceeds to compute:

g*_0(0) + g*_0(1) = 2

This is easy because g*_0 is univariate of degree at most 2.

V sees that P's claims so far are consistent.

V selects 189 uniformly at random from GF(331), independently of 

<a id='complexity'></a>
## Complexity and proof size
↑↑ [Contents](#contents) ↑ [Why V should be convinced if P passes sum check](#why) ↓ [References](#references)

<a id='verifier_runtime'></a>
### Verifier's runtime
↓ [Verifier's space complexity](#verifier_space)

In Round $j$ of the sum-check protocol, verifier V performs the following steps:
1. Verify that $\mathrm{deg}(g_j) \le \mathrm{deg}_j(g)$.
2. Verify that $g^*_{j - 1}(r_{j - 1}) = g^*_j(0) + g^*_j(1)$ (substitute $H^*$ for $g^*_{j-1}(r_{j-1})$ in Round $0$).
3. Generate a random field element $r_j$.
4. In the final round, V verifies that $g_{v-1}(r_{v-1}) = g(r_0,\ldots,r_{v-1}$).

Note that $\mathrm{deg}_j(g) \le d$ where $d = \mathrm{deg}(g)$. Since each $g^*_j$ is a univariate polynomial, the time taken in Round $j$ is:

1. **Degree verification:** $O(d)$, assuming P sends only the nonzero coefficients of $g_j$, i.e. the set $\{(i,c_i) : c_i \ne 0\}$, where $c_i$ is the coefficient of $X_j^i$ in $g_j$.
2. **Evaluation check:** $O(d)$, because evaluating a univariate polynomial of degree $d$ can be done with $d$ multiplications and $d$ additions, using Horner's method.
3. **Random field element generation:** $O(1)$ if precomputed 'offline' before the start of the protocol; $O(k\log p)$ if performed 'online', where $p^k$ is the order of $\mathbb{F}$.
4. **Final round verification:** $O(d)$ to compute $g_{v-1}(r_{v-1})$, plus the time it takes to compute or obtain (e.g. from an oracle) the value of $g(r_0,\ldots,r_{v-1})$.

**Horner's method** is an efficient way to evaluate a polynomial, given its coefficients. We rewrite a degree $d$ polynomial 
\begin{equation*}
 P(X) = c_0 + c_1X + \cdots + c_{d}X^{d}
\end{equation*}
in nested form:
\begin{equation*}
P(X) = c_0 + X(c_1 + c_2(X + c_3(X + \cdots + X(c_{d-1} + Xc_d)\cdots ))
\end{equation*}
Starting with $a_d := c_d$, compute recursively:
\begin{equation*}
a_j = c_j + aa_{j + 1}, \quad j = d - 1, d - 2,\ldots,0.
\end{equation*}
The result is $P(a) = a_0$.

**Random element generation** proceeds as follows. If $k = 1$ each field element is an integer in $\{0,\ldots,p - 1\}$. Generate a random bitstring of length $m$, where $2^{m - 1} < p \le 2^m$. If the bitstring represents an integer in $\{0,\ldots,p-1\}$, accept it; otherwise, reject it and retry. If $k > 1$, represent field elements polynomials of degree $k - 1$ with coefficients in $\mathbb{F}_p$. Random coefficients are selected via the same bitstring process.

In practical implementations, $d$ is usually much smaller than $k\log p$, so random number generation dominates the complexity if done online. To mitigate this, $r_0,\ldots,r_{v-1}$ are typically precomputed. We assume this is the case.

**Complexity of evaluating $g$ at a random point.** The most time-consuming part of the process is computing $g(r_0,\ldots,r_{v-1})$, a $v$-variate polynomial of degree $d$. Such a polynomial has at most
\begin{equation*}
\binom{v + d}{d} = \frac{(v + d)(v + d - 1)\cdots (v + 1)}{d!} \le \frac{(v + d)^d}{d!}
\end{equation*}
nonzero terms: one for each exponent tuple $(e_0,\ldots,e_{v-1})$ with $e_0 + \ldots + e_{v - 1} \le d$. 

To see this, note that the solutions $(e_0,\ldots,e_{v-1}) \in \mathbb{N}^v$ to $e_0 + \cdots + e_{v - 1} \le d$ are in one-to-one correspondence with the solutions $(e_0,\ldots,e_v) \in \mathbb{N}^{v + 1}$ to $e_0 + \cdots + e_v = d$. This, in turn, corresponds to the number of ways to arrange $d$ 'stars' and $v$ 'bars' in $d + v$ positions. 

For example, $|*||***|*$ corresponds to $e_0 = 1$, $e_1 = 0$, $e_2 = 3$, $e_3 = 1$, satisying $e_0 + e_1 + e_2 + e_3 = 5$. Hence, $e_0 + e_1 + e_2 = 4 \le 5$, representing the term $X_0X_2^3$ in a $3$-variate polynomial of total degree $5$.

Using a generalized Horner's method, each term requires $O(v)$ operations to evaluate, resulting in a total complexity of 
\begin{equation*}
O\left(\binom{v + d}{d}\cdot v\right).
\end{equation*}
Since 
\begin{equation*}
\binom{v + d}{d} = \frac{(v + d)(v + d - 1)\cdots (v + 1)}{d!} \le \frac{(v + d)^d}{d!},
\end{equation*}
the complexity reduces to $O(v^{d + 1})$ for bounded $d$. (In practical implementations, $d$ is typically $2$ or $3$.)

**Verifier's runtime.** However, $g(r_0,\ldots,r_{v-1})$ is not computed by V. V's total runtime for all rounds is:
\begin{equation*}
O(vd).
\end{equation*}

<a id='verifier_space'></a>
### Verifier's space complexity
↑ [Verifier's runtime](#verifier_runtime) ↓ [Prover's runtime](#proof_runtime)

The verifier generates and stores $v$ random field elements $r_0, \ldots, r_{v-1}$, all of which must be retained because $g(r_0, \ldots, r_{v-1})$ is evaluated and compared to $g_{v-1}(r_{v-1})$ in the final round.

In each round $j$ ($0 \leq j \leq v - 1$), the verifier uses $O(d+1)$ field elements while applying Horner's method to evaluate $g^*_{j-1}(r{j-1})$, $g^*_j(0)$, and $g^*_j(1)$. Each polynomial $g_j$ is represented by either $O(d+1)$ coefficients or $O(d+1)$ points, and each polynomial is required for at most two consecutive rounds.

The verifier also keeps track of the degree of the initial polynomial, which requires $O(1)$ space for a single field element or integer.

Thus, in terms of field elements, the total space complexity for the verifier is $O(v + d)$. If the field $\mathbb{F}_{p^k}$ is used, where each element is represented using $O(k \log p)$ bits, the total space complexity is $O\left((v + d) k \log p\right)$ bits.

<a id='prover_runtime'></a>
### Prover's runtime
↑ [Verifier's space complexity](#verifier_space) ↓ [Prover's space complexity](#prover_space)

In Round 0, an honest prover P computes:
\begin{equation*}
 g_0(X_0) := \sum_{b_{1}  \in  \{0,1\}} \cdots \sum_{b_{v - 1} \in \{0,1\}} g(X_0,b_{1},\ldots,b_{v-1}),
\end{equation*}
As already discussed, Horner's method facilitates its evaluation at $(b_{1},\ldots,b_{v - 1})$ using a number of operations that is at most of order
\begin{equation*}
 \binom{v + d}{d}\cdot v.
\end{equation*}
Evaluating $g$ at each point $(b_0,\ldots,b_{v-1})$ using Horner's method requires at most: 
\begin{equation*}
O\left(\binom{v + d}{d}\cdot v \right)
\end{equation*}
operations per point. Since there are $2^{v-1}$ points in the Boolean hypercube $\{0,1\}^{v-1}$, the total cost in Round 0 is:
\begin{equation*}
O\left(\binom{v + d}{d}\cdot v\cdot 2^v \right).
\end{equation*}
In Round $j$, $1 \le j \le v - 1$, an honest prover P computes:
\begin{equation*}
 g_j(X_j) := \sum_{b_{j + 1}  \in  \{0,1\}} \cdots \sum_{b_{v - 1} \in \{0,1\}} g(r_0,\ldots,r_{j-1},X_j,b_{j + 1},\ldots,b_{v-1}).
\end{equation*}
Similarly, the computation for $g_j(X_j)$ involves at most
\begin{equation*}
 \binom{v - j + d}{d}\cdot (v - j)\cdot 2^{v - j} \le \binom{v + d}{d}\cdot v \cdot 2^{v - j}
\end{equation*}
operations. Summing across all rounds, the total runtime is:
\begin{equation*}
 \sum_{j = 0}^{v - 1} O\left(\binom{v + d}{d}\cdot v\cdot 2^{v - j} \right)
\end{equation*}
Using the geometric series sum $2^v + 2^{v - 1} + \cdots + 2 = 2^{v + 1} - 2$, the total runtime simplifies to:
\begin{equation*}
O\left(\binom{v + d}{d}\cdot v\cdot 2^v\right).
\end{equation*}
This further simplifies to:
\begin{equation*}
O\left(\frac{v(v + d)^d2^v}{d!}\right).
\end{equation*}
If $d$ is bounded, as it typically is in practical implementations, the runtime becomes 
\begin{equation*}
O(v^{d + 1}2^v),
\end{equation*}
dominated by the exponential term $2^v$, which arises from the size of the Boolean hypercube.

<a id='prover_space'></a>
### Prover's space complexity
↑ [Prover's runtime](#prover_runtime) ↓ [Proof size](#proof_size)

P's space complexity is dominated by the need to store the multivariate polynomial $g(X_0, \ldots, X_{v-1})$, which may have up to $\displaystyle\binom{v + d}{d}$ nonzero terms. Each term requires storing a coefficient and an associated exponent tuple, resulting in a total space requirement of 
\begin{equation*}
O\left(\binom{v + d}{d}\right)
\end{equation*}
field elements for the polynomial representation. 

During each round of the protocol, the prover computes a univariate polynomial $g_j(X_j)$ by summing over the Boolean hypercube. This involves evaluating $g(X_0, \ldots, X_{v-1})$ at $2^{v-j-1}$ points. While summing, P does not need to store all intermediate evaluations. Instead, it requires space to accumulate the sum and temporarily store the values needed for evaluating $g$ at a single point. Using Horner's method, evaluating $g$ at one point requires 
\begin{equation*}
O\left(\binom{v + d}{d}\cdot v\right) 
\end{equation*}
operations and the same space to store intermediate results. Therefore, the space required for hypercube evaluations is proportional to \begin{equation*}
O\left(\binom{v + d}{d}\cdot v\right),
\end{equation*}
which is the same order as the storage for $g$.

Additionally, the prover must store the univariate polynomial $g_j(X_j)$ during each round. Since $g_j(X_j)$ has degree at most $d$, it can be represented by $O(d+1)$ field elements. This contribution is small compared to the storage requirements for $g(X_0, \ldots, X_{v-1})$ and hypercube summations. As a result, the total space complexity for the prover is dominated by 
\begin{equation*}
O\left(\displaystyle\binom{v + d}{d}\cdot v\right) = O\left(\frac{v(v + d)^d}{d!}\right),
\end{equation*}
which simplifies to $O\left(v^{d+1}\right)$ for bounded $d$. If the field $\mathbb{F}_{p^k}$ is used, the bit complexity becomes $O\left(v^{d+1} k \log p\right)$.

<a id='proof_size'></a>
### Proof size
↑ [Prover's space complexity](#prover_space)

The **proof size** is the total amount of information communicated from the prover to the verifier during the sum-check protocol. It is a critical metric for assessing the protocol's efficiency, especially in applications like zero-knowledge proofs or other cryptographic systems.

In Round $j$, $0 \le j \le v - 1$, P sends a univariate polynomial of degree at most $d$ to V. This can be done by sending $d + 1$ field elements as coefficients of the polynomial, or $(d + 1)$ points on the polynomial (which V can then interpolate). Thus, in terms of **field elements**, the total proof size is 
\begin{equation*}
O(vd)
\end{equation*}
(we can safely assume $d > 0$, so no need to write $O(v(d + 1))$). In terms of **bits**, since each element of $\mathbb{F}_{p^k}$ can be expressed as a bitstring of length $O(k\log p)$, the total proof size is 
\begin{equation*}
O(vdk\log p).
\end{equation*}

<a id='references'></a>
## References
↑↑ [Contents](#contents) ↑ [Complexity and proof size](#complexity)

<a id='dml1978'></a>
[DML1978] DeMillo, R. A. and R. Lipton. '[A probabilistic remark on algebraic program testing](https://doi.org/10.1016/0020-0190(78)90067-4).' _Information Processing Letters_ 7(4):193&ndash;195, 1978.

<a id='lfkn1992'></a>
[LFKN1992] Lund, C., L. Fortnow, K. Howard, and N. Nisan. '[Algebraic methods for interactive proof systems](https://doi.org/10.1145/146585.146605).' _Journal of the ACM_, 39(4):859&ndash;868, 1992.


<a id='ore1922'></a>
[ORE1922] Ore, Ö. '[Über höhere Kongruenzen]().' _Norsk Matematisk Forenings skrifter Ser. I_ (1922), no. 7, 15 pages.

<a id='sch1980'></a>
[SCH1980] Schwartz, J. T. '[Fast probabilistic algorithms for verification of polynomial identities](https://doi.org/10.1145/322217.322225).' _Journal of the ACM_, 27(4):701&ndash;717, 1980.

<a id='tha2015'></a>
[THA2015] Thaler, J. '[A note on the GKR protocol](https://api.semanticscholar.org/CorpusID:16402332).' 2015.

<a id='zip1979'></a>
[ZIP1979] Zippel, R. '[Probabilistic algorithms for sparse polynomials](https://doi.org/10.1007/3-540-09519-5_73).' In Ng, E. W. (ed.). _Symbolic and Algebraic Computation, EUROSAM '79, An International Symposiumon Symbolic and Algebraic Computation, Marseille, France, June 1979, Proceedings_. Lecture Notes in Computer Science. Vol. 72. Springer. pp. 216&ndash;226.